In [1]:
import os
if '_cwd_set' not in locals(): locals()['_cwd_set'] = os.chdir(os.path.dirname(os.getcwd()))
import omnifig as fig
fig.initialize()
from src.jimports import *
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, precision_recall_curve
# for all stats at once
from sklearn.metrics import precision_recall_fscore_support
from src.util import set_default_device, repo_root, data_root
from src.interventions import ClassLevelLabelIntervention
from src.dataset import RawCOCO, SimpleCOCO, RawCOCOCaptions, COCOCaptions, COCO, MNIST
from src.interventions import ClassLevelModuleIntervention, ClassLevelCentroidIntervention, ClassLevelLinearIntervention
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score
from contextlib import redirect_stdout
from sklearn.neighbors import KernelDensity
from scipy.optimize import root_scalar
from scipy.stats import norm
from src.baselines import ConceptAlgebra
from matplotlib import patches as mpatches
from sklearn.metrics import precision_recall_fscore_support

device = set_default_device();

Selecting CUDA device 0 with 48386 MiB free memory and 0% utilization


In [2]:

def roc_threshold(positives, negatives):
    """
    Compute the optimal threshold using the ROC curve.
    
    Args:
        positives: Array of data points from the positive class.
        negatives: Array of data points from the negative class.

    Returns:
        Optimal threshold (float) based on the ROC curve.
    """
    y_true = np.concatenate([np.ones(len(positives)), np.zeros(len(negatives))])
    y_scores = np.concatenate([positives, negatives])
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    optimal_idx = np.argmax(tpr - fpr)
    return thresholds[optimal_idx]

def bayes_threshold(positives, negatives):
    """
    Compute the Bayes threshold using the intersection of Gaussian PDFs.
    Assumes both distributions are Gaussian.

    Args:
        positives: Array of data points from the positive class.
        negatives: Array of data points from the negative class.

    Returns:
        Bayes threshold (float).
    """
    m1, s1 = np.mean(positives), np.std(positives)
    m0, s0 = np.mean(negatives), np.std(negatives)

    def pdf_diff(x):
        return norm.pdf(x, loc=m1, scale=s1) - norm.pdf(x, loc=m0, scale=s0)

    result = root_scalar(pdf_diff, bracket=[m0, m1], method='brentq')
    return result.root


In [3]:
# fullddata = COCO(eval_split=-0.1)
syscfg = fig.create_config('h/ws2')
syscfg.silent=True


In [4]:
# loc = '/data/felix/cache/checkpoints/vae128_coco_20250116_222421/ckpt_100000'
# loc = '/data/felix/cache/checkpoints/ae128_coco_20250114_230254/ckpt_020000/'
# loc = '/data/felix/cache/checkpoints/sae128_coco_20250118_213955/ckpt_100000'
# loc = '/ssd/felix/cache/checkpoints/sae512_coco_20250128_121655/ckpt_010000'
# loc = Path(loc)

In [5]:
# cfg = fig.create_config(*'h/ws2 a/wide norm m/ced d/coco-img'.split(), **{'latent-dim': 512, 'classifier.dropout': 0.1})
# trainer = cfg.pull('trainer', silent=True)
# model = trainer.model
# traindataset = cfg.pull('dataset', silent=True)
# valdataset = traindataset.as_eval()
# traindataset.prepare(device=device);
# valdataset.prepare(device=device)
# system = Structured(traindataset, *trainer.gadgetry())
# system.mechanize() # sync for gears and spaces
# mech = system.mechanics()
# model.prepare(device=device);
# for p in model.parameters():
#     p.requires_grad = False
# model.load_checkpoint(path=loc.joinpath('model'));

In [39]:
ckptroot = '/ssd/felix/cache/checkpoints/'

textckpts = {
    # 'sae512_cococap_20250129_105634': 'concept-text', # vital-deluge-11
    # 'sae512_cococap_20250125_123713': 'text1-module', # treasured-sunset-9
    'sae512_cococap_20250125_114125': 'text2-module', # glad-butterfly-15
    'sae512_cococap_20250129_115412': 'text2-fromtext-module', # fragrant-armadillo-23

    # 'sae512_cococap_20250129_114401': 'text1-centroid', # dutiful-shape-12
    # 'sae512_cococap_20250129_114559': 'text1-affine', # zany-sea-13
    
    # 'sae512_cococap_20250129_114725': 'sem-text1-module', # crimson-salad-14
    # 'sae512_cococap_20250129_115122': 'sem-text1-centroid', # usual-cloud-16
    # 'sae512_cococap_20250129_115155': 'sem-text1-affine', # usual-serenity-17
}

imgckpts = {
    # 'sae512_coco_20250128_121655': 'concept-img', # light-wood-123
    # 'sae512_coco_20250124_164611': 'img1-module', # super-deluge-104
    'sae512_coco_20250129_114055': 'img2-module', # distinctive-disco-3
    'sae512_coco_20250125_234035': 'img2-fromimg-module', # serene-leaf-1

    # 'sae512_coco_20250129_112704': 'img1-centroid', # vocal-wildflower-124
    # 'sae512_coco_20250129_112805': 'img1-affine', # scarlet-flower-125
    
    # 'sae512_coco_20250129_112925': 'sem-img1-module', # jumping-sun-126
    # 'sae512_coco_20250129_113004': 'sem-img1-centroid', # kind-darkness-127
    # 'sae512_coco_20250129_113041': 'sem-img1-affine', # rare-fog-128
    
}


modality = 'img'
# modality = 'text'
assert modality in ['img', 'text']
if modality == 'img':
    confname = 'd/coco-img'
    ckpts = imgckpts
    ckptname = 'ckpt_020000'
    ckptname = 'ckpt_080000'
    # ckptname = 'ckpt_100000'
else:
    confname = 'd/coco-cap'
    ckpts = textckpts
    ckptname = 'ckpt_080000'
    # ckptname = 'ckpt_100000'

assert all(Path(ckptroot).joinpath(k,ckptname).exists() for k in ckpts.keys()), f'missing: {", ".join(k for k in ckpts.keys() if not Path(ckptroot).joinpath(k,ckptname).exists())}'
print(f'{modality} {len(ckpts)} models: {", ".join(ckpts.values())}')

img 2 models: img2-module, img2-fromimg-module


In [40]:
traindataset = fig.create_config('h/ws2', confname, split='train', eval_split=0.1).pull('dataset', silent=True)
valdataset = traindataset.as_eval().prepare(device=device)
# traindataset.prepare(device=device);
# todo prepare validation samples
num_classes = valdataset.label_space.n
class_names = valdataset.label_space.class_names

fiids = valdataset.get_image_id(np.arange(valdataset.size))
valmap = {}
for i, im in enumerate(fiids.tolist()):
    valmap.setdefault(im, []).append(i)
valpicks = np.array([options[0] for key, options in valmap.items()])
traindataset, valdataset, len(valpicks)

(SimpleCOCO[106459](index, image, image_id, observation, instances, label),
 SimpleCOCO[11828](index, image, image_id, observation, instances, label),
 11828)

In [41]:
def load_model(name):
    path = Path(ckptroot).joinpath(name, ckptname)

    cfg = fig.create_config(str(path/'config.yaml'), **{'use-wandb': False, 'device': device})
    cfg.silent=True

    valdataset._mechanics = None
    with redirect_stdout(None):
        trainer = cfg.pull('trainer', silent=True)
        trainer.setup(valdataset, device=device)
        valdataset._mechanics = None

    model = trainer.model
    for p in model.parameters():
        p.requires_grad = False
    model.load_checkpoint(path=path.joinpath('model'))
    model.eval()

    intv = getattr(trainer, 'intervention', None)
    if intv is None:
        assert ckpts[name].startswith('concept-'), f'no intervention for {name} ({ckpts[name]})'
        intv = ConceptAlgebra(as_delta=False)
        trainer.intervention = intv
    else:
        for p in intv.parameters():
            p.requires_grad = False
        intv.load_checkpoint(path=path.joinpath('interventions'))
        intv.eval()
        intv.to(device)

    trainer.ckptname = name
    trainer.ident = ckpts[name]
    return trainer
trainers = [load_model(name) for name in tqdm(ckpts.keys())]

  0%|          | 0/2 [00:00<?, ?it/s]

/home/felix/workspace/clones/omni-learn/omnilearn/machines.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(path, map_location='cpu')
/home/felix/works

In [42]:
vpicks = valpicks#[:10000]
valbatch = Context(valdataset, DictGadget({'size': len(vpicks), 'index': vpicks}))
valbatch, valbatch['observation'].shape

(Context(index, observation, {image}, {image_id}, {instances}, {label}, {size}),
 torch.Size([11828, 768]))

In [43]:
# compute classification thresholds
for tr in tqdm(trainers):
    vz = tr.model.encode(valbatch['observation'])
    if isinstance(tr.intervention, ConceptAlgebra):
        tr.intervention.extract_projections(vz, valbatch['label'])
    
    vscore = tr.model.predict(vz)
    vy = valbatch['label']

    thresholds = []
    aucs = []
    f1s = []
    for ci in range(num_classes):        
        points = vscore[:, ci].cpu().numpy()
        gtlabels = vy[:, ci].cpu().numpy()

        threshold = roc_threshold(points[gtlabels], points[~gtlabels])
        # threshold = bayes_threshold(points[gtlabels], points[~gtlabels])
        thresholds.append(threshold)
        f1 = f1_score(gtlabels, points > threshold)
        f1s.append(f1)
        aucs.append(roc_auc_score(gtlabels, points))
    
    tr.aucs = torch.as_tensor(aucs).to(device)
    tr.f1s = torch.as_tensor(f1s).to(device)
    tr.thresholds = torch.as_tensor(thresholds).to(device)
print(tabulate([[t.ident, t.aucs.mean().item(), t.aucs.min().item(), t.f1s.mean().item(), t.f1s.min().item()] for t in trainers], headers=['model', 'val-avg-auc', 'val-min-auc', 'val-avg-f1', 'val-min-f1']))

  0%|          | 0/2 [00:00<?, ?it/s]

model                  val-avg-auc    val-min-auc    val-avg-f1    val-min-f1
-------------------  -------------  -------------  ------------  ------------
img2-module               0.940402       0.81844       0.369796     0.0155678
img2-fromimg-module       0.942137       0.803919      0.380324     0.0578662


In [44]:
num = 5
tbl = []
for tr in trainers:
    order = np.argsort(tr.aucs.cpu().numpy())
    top_auc = [f'{class_names[i]}:{tr.aucs[i].item():.3f}' for i in order[:-num-1:-1]]
    low_auc = [f'{class_names[i]}:{tr.aucs[i].item():.3f}' for i in order[num-1::-1]]

    order = np.argsort(tr.f1s.cpu().numpy())
    top_f1 = [f'{class_names[i]}:{tr.f1s[i].item():.3f}' for i in order[:-num-1:-1]]
    low_f1 = [f'{class_names[i]}:{tr.f1s[i].item():.3f}' for i in order[num-1::-1]]

    # tbl.append([tr.ident, '\n'.join(top_auc), '\n'.join(low_auc), '\n'.join(top_f1), '\n'.join(low_f1)])
    tbl.append([tr.ident, 
                '\n'.join(tabulate([x.split(':') for x in top_auc]).split('\n')[1:]), 
                '\n'.join(tabulate([x.split(':') for x in low_auc]).split('\n')[1:]),
                '\n'.join(tabulate([x.split(':') for x in top_f1]).split('\n')[1:]),
                '\n'.join(tabulate([x.split(':') for x in low_f1]).split('\n')[1:])])
tbl = [['model', 'top-auc', 'low-auc', 'top-f1', 'low-f1']] + tbl
ttbl = list(zip(*tbl))
print('validation class rankings')
print(tabulate(ttbl[1:], headers=ttbl[0]))

validation class rankings
model    img2-module            img2-fromimg-module
-------  ---------------------  ---------------------
top-auc  zebra           0.998  zebra     0.998
         giraffe         0.997  airplane  0.995
         baseball glove  0.995  giraffe   0.994
         baseball bat    0.995  bear      0.993
         elephant        0.993  elephant  0.993
         --------------  -----  --------  -----
low-auc  bench         0.856    bench         0.845
         potted plant  0.854    cup           0.833
         cell phone    0.848    potted plant  0.82
         handbag       0.823    handbag       0.818
         backpack      0.818    backpack      0.804
         ------------  -----    ------------  -----
top-f1   tennis racket  0.865   person         0.877
         person         0.853   tennis racket  0.83
         elephant       0.813   train          0.79
         giraffe        0.789   airplane       0.788
         airplane       0.697   elephant       0.757
      

In [45]:
testdataset = fig.create_config('h/ws2', confname, split='val').pull('dataset', silent=True).prepare(device=device)
fiids = testdataset.get_image_id(np.arange(testdataset.size))
idmap = {}
for i, im in enumerate(fiids.tolist()):
    idmap.setdefault(im, []).append(i)
picks = np.array([options[0] for options in idmap.values()])
testdataset, len(picks)

(SimpleCOCO[5000](index, image, image_id, observation, instances, label), 5000)

In [46]:
batch = Context(testdataset, DictGadget({'size': len(picks), 'index': picks}))
batch, batch['size']

(Context(size, {index}, {image}, {image_id}, {observation}, {instances}, {label}),
 5000)

In [47]:
gti = ClassLevelLabelIntervention()
def compute_interventional_metrics(y, yhat, iy, iyhat):
    valid = y == yhat
    changed_mask = iy != y
    assert changed_mask.any(), f'nothing changed'
    
    correct = iy == iyhat

    sel = changed_mask
    sel = changed_mask & valid
    c_pre, c_rec, c_f1, _ = precision_recall_fscore_support(torch.ones_like(correct[sel]).cpu().numpy(), correct[sel].cpu().numpy(), average='binary')
    # c_pre, c_rec, c_f1, _ = precision_recall_fscore_support(iy[sel].cpu().numpy(), iyhat[sel].cpu().numpy(), average='binary')
    # c_pre, c_rec, c_f1 = 0., 0., 0.
    c_acc = correct[sel].float().mean().item()

    sel = ~changed_mask
    sel = ~changed_mask & valid
    u_pre, u_rec, u_f1, _ = precision_recall_fscore_support(iy[sel].cpu().numpy(), iyhat[sel].cpu().numpy(), average='binary')
    # u_acc = correct[sel].float().mean().item()
    # balanced accuracy = 1/2 * ((TP/(TP+FN)) + (TN/(TN+FP)))
    tp = iy[sel] & iyhat[sel]
    tn = ~iy[sel] & ~iyhat[sel]
    fp = ~iy[sel] & iyhat[sel]
    fn = iy[sel] & ~iyhat[sel]
    u_acc = 0.5 * ((tp.sum().float() / (tp.sum() + fn.sum())) + (tn.sum().float() / (tn.sum() + fp.sum())))

    return c_pre, c_rec, c_f1, c_acc, u_pre, u_rec, u_f1, u_acc

In [48]:
for tr in tqdm(trainers):
    x = batch['observation']
    y = batch['label']
    z = tr.model.encode(x)

    score = tr.model.predict(z)
    yhat = score > tr.thresholds.unsqueeze(0)

    obs_auc = []
    obs_pres, obs_recs, obs_f1s = [], [], []
    obs_accs = []
    for ic in range(num_classes):
        auc = roc_auc_score(y[:, ic].cpu().numpy(), score[:, ic].cpu().numpy())
        obs_auc.append(auc)
        f1 = f1_score(y[:, ic].cpu().numpy(), yhat[:, ic].cpu().numpy())
        # pre, rec, f1, _ = precision_recall_fscore_support(y[:, ic].cpu().numpy(), yhat[:, ic].cpu().numpy(), average='binary')
        # obs_pres.append(pre.item())
        # obs_recs.append(rec.item())
        obs_f1s.append(f1.item())
        # balanced accuracy
        tp = y[:, ic] & yhat[:, ic]
        tn = ~y[:, ic] & ~yhat[:, ic]
        fp = ~y[:, ic] & yhat[:, ic]
        fn = y[:, ic] & ~yhat[:, ic]
        acc = 0.5 * ((tp.sum().float() / (tp.sum() + fn.sum())) + (tn.sum().float() / (tn.sum() + fp.sum())))
        obs_accs.append(acc.item())
    obs_auc = np.array(obs_auc)
    # obs_pres = np.array(obs_pres)
    # obs_recs = np.array(obs_recs)
    obs_f1s = np.array(obs_f1s)
    obs_accs = np.array(obs_accs)
    # print(obs_auc.mean(), obs_auc.min(), obs_f1s.mean(), obs_f1s.min(), obs_pres.mean(), obs_recs.mean(), obs_sup.mean())
    # print(obs_auc.mean(), obs_auc.min(), obs_f1s.mean(), obs_f1s.min())
    tr.obs_auc = torch.as_tensor(obs_auc).to(device)
    # tr.obs_pres = torch.as_tensor(obs_pres).to(device)
    # tr.obs_recs = torch.as_tensor(obs_recs).to(device)
    tr.obs_f1s = torch.as_tensor(obs_f1s).to(device)
    tr.obs_accs = torch.as_tensor(obs_accs).to(device)

    def run_interventions(adds, rems):
        if isinstance(tr.intervention, ClassLevelModuleIntervention):
            cond = tr.intervention.get_conditioning(adds, rems)
            iz = tr.intervention.apply_intervention(z, cond)
        else:
            iz = tr.intervention.apply_intervention(z, adds, rems)
        # iz = Context(tr.intervention, DictGadget({'ambient': z, 'latent_d': z, 'add_class': adds, 'remove_class': rems}))['probe']
        iscore = tr.model.predict(iz)
        iyhat = iscore > tr.thresholds.unsqueeze(0)
        iy = gti.apply_intervention(y, adds, rems)
        
        c_pre, c_rec, c_f1, c_acc, u_pre, u_rec, u_f1, u_acc = compute_interventional_metrics(y, yhat, iy, iyhat)
        return c_pre, c_rec, c_f1, c_acc, u_pre, u_rec, u_f1, u_acc
    
    torch.manual_seed(11)

    num_intv_per_sample = 10

    c_accs, u_accs = [], []
    c_f1s, c_pres, c_recs, c_supps = [], [], [], []
    u_f1s, u_pres, u_recs, u_supps = [], [], [], []
    for _ in range(num_intv_per_sample):
        adds, rems = None, None
        adds = gti.sample_add_intervention(y)
        rems = gti.sample_remove_intervention(y)
        
        c_pre, c_rec, c_f1, c_acc, u_pre, u_rec, u_f1, u_acc = run_interventions(adds, rems)
        c_accs.append(c_acc); c_f1s.append(c_f1); c_pres.append(c_pre); c_recs.append(c_rec)
        u_accs.append(u_acc); u_f1s.append(u_f1); u_pres.append(u_pre); u_recs.append(u_rec)

    tr.itrain_c_acc = torch.as_tensor(c_accs).to(device).mean().item()
    tr.itrain_c_f1 = torch.as_tensor(c_f1s).to(device).mean().item()
    tr.itrain_u_f1 = torch.as_tensor(u_f1s).to(device).mean().item()
    tr.itrain_u_acc = torch.as_tensor(u_accs).to(device).mean().item()


    # add interventions (not seen during training)
    torch.manual_seed(13)

    c_accs, u_accs = [], []
    c_f1s, c_pres, c_recs, c_supps = [], [], [], []
    u_f1s, u_pres, u_recs, u_supps = [], [], [], []
    for _ in range(num_intv_per_sample):
        adds = gti.sample_add_intervention(y)
        # rems = gti.sample_remove_intervention(y)
        
        c_pre, c_rec, c_f1, c_acc, u_pre, u_rec, u_f1, u_acc = run_interventions(adds, rems)
        c_accs.append(c_acc); c_f1s.append(c_f1); c_pres.append(c_pre); c_recs.append(c_rec)
        u_accs.append(u_acc); u_f1s.append(u_f1); u_pres.append(u_pre); u_recs.append(u_rec)

    tr.iadd_c_acc = torch.as_tensor(c_accs).to(device).mean().item()
    tr.iadd_c_f1 = torch.as_tensor(c_f1s).to(device).mean().item()
    tr.iadd_u_f1 = torch.as_tensor(u_f1s).to(device).mean().item()
    tr.iadd_u_acc = torch.as_tensor(u_accs).to(device).mean().item()

    # remove interventions (not seen during training)
    torch.manual_seed(17)

    num_intv_per_sample = 10

    c_accs, u_accs = [], []
    c_f1s, c_pres, c_recs, c_supps = [], [], [], []
    u_f1s, u_pres, u_recs, u_supps = [], [], [], []

    for _ in range(num_intv_per_sample):
        adds, rems = None, None
        # adds = gti.sample_add_intervention(y)
        rems = gti.sample_remove_intervention(y)
        
        c_pre, c_rec, c_f1, c_acc, u_pre, u_rec, u_f1, u_acc = run_interventions(adds, rems)
        c_accs.append(c_acc); c_f1s.append(c_f1); c_pres.append(c_pre); c_recs.append(c_rec)
        u_accs.append(u_acc); u_f1s.append(u_f1); u_pres.append(u_pre); u_recs.append(u_rec)

    tr.irem_c_acc = torch.as_tensor(c_accs).to(device).mean().item()
    tr.irem_c_f1 = torch.as_tensor(c_f1s).to(device).mean().item()
    tr.irem_u_f1 = torch.as_tensor(u_f1s).to(device).mean().item()
    tr.irem_u_acc = torch.as_tensor(u_accs).to(device).mean().item()

    tr.igen_c_acc = (tr.iadd_c_acc + tr.irem_c_acc) / 2
    tr.igen_c_f1 = (tr.iadd_c_f1 + tr.irem_c_f1) / 2
    tr.igen_u_acc = (tr.iadd_u_acc + tr.irem_u_acc) / 2
    tr.igen_u_f1 = (tr.iadd_u_f1 + tr.irem_u_f1) / 2



  0%|          | 0/2 [00:00<?, ?it/s]

In [49]:
print(tabulate([[t.ident, t.obs_auc.mean().item(), t.obs_auc.min().item(), t.obs_f1s.mean().item(), t.obs_f1s.min().item(), 
                 t.obs_accs.mean().item(), t.obs_accs.min().item(),
                 t.itrain_c_acc, t.itrain_u_acc, t.itrain_c_f1, t.itrain_u_f1,
                #  t.iadd_c_acc, t.iadd_u_acc, t.iadd_c_f1, t.iadd_u_f1,
                #  t.irem_c_acc, t.irem_u_acc, t.irem_c_f1, t.irem_u_f1,
                 t.igen_c_acc, t.igen_u_acc, t.igen_c_f1, t.igen_u_f1
                 ] for t in trainers], headers=['model', 'obs-avg-auc', 'obs-min-auc', 'obs-avg-f1', 'obs-min-f1', 
                                                'obs-avg-acc', 'obs-min-acc',
                                                'itrain-c-acc', 'itrain-u-acc', 'itrain-c-f1', 'itrain-u-f1',
                                                # 'iadd-c-acc', 'iadd-u-acc', 'iadd-c-f1', 'iadd-u-f1',
                                                # 'irem-c-acc', 'iadd-u-acc', 'irem-c-f1', 'irem-u-f1', 
                                                'igen-c-acc', 'igen-u-acc', 'igen-c-f1', 'igen-u-f1'
                                                ]))

model                  obs-avg-auc    obs-min-auc    obs-avg-f1    obs-min-f1    obs-avg-acc    obs-min-acc    itrain-c-acc    itrain-u-acc    itrain-c-f1    itrain-u-f1    igen-c-acc    igen-u-acc    igen-c-f1    igen-u-f1
-------------------  -------------  -------------  ------------  ------------  -------------  -------------  --------------  --------------  -------------  -------------  ------------  ------------  -----------  -----------
img2-module               0.938733       0.81007       0.368692     0.0169133       0.875328       0.736687        0.803269        0.933648       0.890899       0.47295       0.493179      0.918613     0.601187     0.437806
img2-fromimg-module       0.941325       0.799055      0.37693      0.0538642       0.878109       0.712241        0.707211        0.925563       0.828496       0.301869      0.478037      0.9068       0.613792     0.27934


In [ ]:

# model                obs-avg-auc    obs-min-auc    obs-avg-f1    obs-min-f1    obs-avg-acc    obs-min-acc    itrain-c-acc    itrain-u-acc    itrain-c-f1    itrain-u-f1    igen-c-acc    igen-u-acc    igen-c-f1    igen-u-f1
# -----------------  -------------  -------------  ------------  ------------  -------------  -------------  --------------  --------------  -------------  -------------  ------------  ------------  -----------  -----------
# concept-img             0.962416       0.878639      0.429639     0.028777        0.904266       0.800311        0.894902        0.529389       0.944535      0.0646658      0.947606      0.486482     0.972353    0.0387073
# img1-centroid           0.963831       0.884694      0.425504     0.0411311       0.908353       0.79451         0.995684        0.686866       0.997837      0.104248       0.962352      0.705543     0.980513    0.0940431
# img1-affine             0.964132       0.885492      0.427893     0.0300501       0.907632       0.78964         0.99483         0.869269       0.997408      0.437516       0.995406      0.891964     0.997698    0.372178
# img1-module             0.963528       0.879533      0.426361     0.0428954       0.905049       0.780744        0.745399        0.944005       0.854129      0.395806       0.518853      0.92743      0.653518    0.359271
# sem-img1-module         0.964134       0.889826      0.422297     0.0632411       0.908413       0.800464        0.762864        0.94793        0.86548       0.39603        0.489777      0.932898     0.611343    0.361691
# sem-img1-centroid       0.963952       0.890605      0.423439     0.045           0.908165       0.786379        0.986376        0.744374       0.993141      0.105887       0.895308      0.737202     0.942253    0.0948086
# sem-img1-affine         0.963005       0.88686       0.418932     0.0384615       0.906048       0.798187        0.983091        0.917235       0.991473      0.394326       0.982356      0.920378     0.991099    0.337387

# model                 obs-avg-auc    obs-min-auc    obs-avg-f1    obs-min-f1    obs-avg-acc    obs-min-acc    itrain-c-acc    itrain-u-acc    itrain-c-f1    itrain-u-f1    igen-c-acc    igen-u-acc    igen-c-f1    igen-u-f1
# ------------------  -------------  -------------  ------------  ------------  -------------  -------------  --------------  --------------  -------------  -------------  ------------  ------------  -----------  -----------
# concept-text             0.945094       0.813787      0.397579     0.0167364       0.883867       0.710278        0.881591        0.51173        0.937066      0.04619        0.904767      0.439124     0.949813    0.0316822
# text1-centroid           0.951057       0.836843      0.402163     0.04811         0.889618       0.737239        0.826898        0.680325       0.905246      0.0946115      0.701129      0.719687     0.817994    0.0967813
# text1-affine             0.950664       0.831114      0.403007     0.0666667       0.890149       0.744445        0.873461        0.872696       0.932456      0.330226       0.763792      0.885963     0.861547    0.287484
# text1-module             0.950841       0.843812      0.401961     0.0255941       0.890725       0.753219        0.8167          0.946388       0.899101      0.510001       0.506382      0.926649     0.613678    0.481651
# sem-text1-module         0.95062        0.836037      0.413715     0.025641        0.889725       0.746772        0.790387        0.951987       0.882921      0.532646       0.496927      0.928101     0.610615    0.487717
# sem-text1-centroid       0.950422       0.837426      0.404985     0.025641        0.889365       0.743982        0.862517        0.926724       0.926181      0.430344       0.58371       0.839321     0.696554    0.263238
# sem-text1-affine         0.951138       0.837904      0.402504     0.029661        0.889553       0.762687        0.897039        0.922678       0.945724      0.455885       0.787177      0.914599     0.876818    0.349629

# model                  obs-avg-auc    obs-min-auc    obs-avg-f1    obs-min-f1    obs-avg-acc    obs-min-acc    itrain-c-acc    itrain-u-acc    itrain-c-f1    itrain-u-f1    igen-c-acc    igen-u-acc    igen-c-f1    igen-u-f1
# -------------------  -------------  -------------  ------------  ------------  -------------  -------------  --------------  --------------  -------------  -------------  ------------  ------------  -----------  -----------
# img2-module               0.938733       0.81007       0.368692     0.0169133       0.875328       0.736687        0.803269        0.933648       0.890899       0.47295       0.493179      0.918613     0.601187     0.437806
# img2-fromimg-module       0.941325       0.799055      0.37693      0.0538642       0.878109       0.712241        0.707211        0.925563       0.828496       0.301869      0.478037      0.9068       0.613792     0.27934

# model                    obs-avg-auc    obs-min-auc    obs-avg-f1    obs-min-f1    obs-avg-acc    obs-min-acc    itrain-c-acc    itrain-u-acc    itrain-c-f1    itrain-u-f1    igen-c-acc    igen-u-acc    igen-c-f1    igen-u-f1
# ---------------------  -------------  -------------  ------------  ------------  -------------  -------------  --------------  --------------  -------------  -------------  ------------  ------------  -----------  -----------
# text2-module                0.929947       0.782453      0.347344     0.0108342       0.865419       0.686714        0.741108        0.936723       0.851305       0.450485      0.470866      0.921789     0.593254     0.423818
# text2-fromtext-module       0.947131       0.834103      0.390687     0.035533        0.886981       0.740133        0.80026         0.945175       0.889048       0.529101      0.505343      0.927797     0.618031     0.494586